In [2]:
import pandas as pd
pd.set_option('display.max_rows', None)  # Set to None for all rows

df = pd.read_csv('glassdoor_salary.csv')



In [3]:
df  = df[~df.eq('--').any(axis=1) & ~df.eq('-1').any(axis=1) & ~df.eq('-1').any(axis=1)]

In [4]:
len(df)

652

In [5]:
salary = df['Salary Estimate'].apply(lambda x : x.split('(')[0])
df['hourly']  = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else  0)
df['employer provided']  = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary' in x.lower() else  0)

In [6]:
#TEMP CELL



In [7]:
minus_kd = salary.apply(lambda x : x.replace('K', '').replace('$', ''))
min_hr = minus_kd.apply(lambda x: x.lower().replace('per hour', '').replace('usd', ''))
#min_hr = min_hr[min_hr.str.contains('-')]

In [8]:
min_hr = pd.concat([df['hourly'],min_hr], axis = 1)


In [9]:
min_hr.loc[(min_hr['hourly'] == 1) & (~min_hr['Salary Estimate'].str.contains('-')), 'Salary Estimate'] = min_hr[(min_hr['hourly'] == 1) & (~min_hr['Salary Estimate'].str.contains(' -'))]['Salary Estimate'].apply(lambda x: str(int(float(x.strip())) - 5) + '-' + x)
min_hr.loc[~min_hr['Salary Estimate'].str.contains('-'), 'Salary Estimate'] = min_hr[~min_hr['Salary Estimate'].str.contains('-')]['Salary Estimate'].apply(lambda x: str(int(float(x.strip())) - 10) + ' -' + x)



In [10]:
#TEMP CELL 



In [11]:
df['min_salary'] = min_hr['Salary Estimate'].apply(lambda x : x.split('-')[0])
df['max_salary'] = min_hr['Salary Estimate'].apply(lambda x:x.split('-')[1])

df['min_salary'] = df['min_salary'].str.strip()
df['min_salary'] = pd.to_numeric(df['min_salary']).astype(int)
df['max_salary'] = df['max_salary'].str.strip()
df['max_salary'] = pd.to_numeric(df['max_salary']).astype(int)


df['avg_salary'] = (df.min_salary + df.max_salary)/2


In [12]:
#TEMP CELL


In [13]:
df = df[df['Location'].str.contains(',')]

In [14]:
df['city'] = df['Location'].apply(lambda x : x.split(',')[0])
df['job_state'] = df['Location'].apply( lambda x : x.split(',')[1])


In [15]:
df['Founded'] = df['Founded'].astype(int)
df['age'] = df.Founded.apply(lambda x : x if x < 1 else 2024-x)

In [16]:
df['python_yn'] = df['Job Description'].apply(lambda x : 1 if 'python' in x.lower() else 0)
df['excel'] = df['Job Description'].apply(lambda x : 1 if 'excel' in x.lower() else 0)
df['tableau'] = df['Job Description'].apply(lambda x : 1 if 'tableau' in x.lower() else 0)
df['spark'] = df['Job Description'].apply(lambda x : 1 if 'spark' in x.lower() else 0)
df['r_yn'] = df['Job Description'].apply(lambda x : 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)

In [17]:
df['desc_len'] = df['Job Description'].apply(lambda x: len(x))

In [18]:
df = df.drop(columns = ['Unnamed: 0'])

In [19]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,...,avg_salary,city,job_state,age,python_yn,excel,tableau,spark,r_yn,desc_len
1,Senior Data Scientist,USD 200K (Employer est.),Title:\r\nSenior Data Scientist for Large Scal...,4.5,Red Dog Media,"Henderson, NV",1 to 50 Employees,2008,Company - Private,Internet & Web Services,...,195.0,Henderson,NV,16,1,0,1,0,0,4484
2,"Manager, Model Validation",USD 76K - 126K (Glassdoor est.),"When you join Sallie Mae, you become a champio...",4.1,Sallie Mae,"Newark, DE",1001 to 5000 Employees,1973,Company - Public,Banking & Lending,...,101.0,Newark,DE,51,1,1,0,0,0,5617
4,Sr. Finance & Data Analyst,USD 64K - 92K (Glassdoor est.),Sr. Finance & Data Analyst\r\nWho is P.A.C.E?\...,3.4,Senior Care Partners PACE,"Battle Creek, MI",201 to 500 Employees,2008,Nonprofit Organization,Health Care Services & Hospitals,...,78.0,Battle Creek,MI,16,0,1,0,0,0,5572
7,"Data Labeling Quality Specialist, Public Secto...",USD 18.00 Per Hour (Employer est.),The Data Labeling Quality Specialist role is a...,3.7,Scale AI,"Saint Louis, MO",51 to 200 Employees,2016,Company - Private,Enterprise Software & Network Solutions,...,15.5,Saint Louis,MO,8,0,1,0,0,0,3788
9,NLP Data Scientist - Python | Machine Learning...,USD 93K - 154K (Glassdoor est.),"In this age of disruption, organizations need ...",4.0,Deloitte,"Tampa, FL",10000+ Employees,1850,Company - Private,Business Consulting,...,123.5,Tampa,FL,174,1,0,1,0,0,4868


In [20]:
df.to_csv('cleaned_df.csv')

In [ ]:
hourly_filt = df['hourly']==1
df.loc[hourly_filt,'min_salary'] *= 3


In [33]:
df.loc[df['hourly']== 1][['min_salary']] = df[df['hourly']== 1][['min_salary']].apply(lambda x: x * 2)
df.loc[df['hourly']== 1][['max_salary']] = df[df['hourly']== 1][['max_salary']].apply(lambda x: x * 2)

C:\Users\dhruv\AppData\Local\Temp\ipykernel_32796\2693997882.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['hourly']== 1][['min_salary']] = df[df['hourly']== 1][['min_salary']].apply(lambda x: x * 2)
C:\Users\dhruv\AppData\Local\Temp\ipykernel_32796\2693997882.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['hourly']== 1][['max_salary']] = df[df['hourly']== 1][['max_salary']].apply(lambda x: x * 2)


In [34]:
df[df['hourly']== 1][['min_salary']]

,min_salary
7,13
56,26
165,70
171,60
172,64
204,63
212,85
222,47
233,80
278,70
